In [42]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json_shot_scraper import flatten_shot, flatten_goal, flatten_complete_pass, flatten_incomplete_pass, flatten_corner
from  player_scraper import flatten_player, flatten_sub
from dataframe_cleaner import (pass_to_shot, corner_to_shot, transpose_coordinates, coord_to_yards, 
                               shot_distance_angle, dummy_columns, drop_own_goals, goal_dummy)
from model_prep import create_hypothetical_df
from transfer_market_scraper import flatten_player_info

In [4]:
pd.set_option('display.max_columns', 50)

In [63]:
from html_scraper import db
from mongo_to_db import game_to_cleaned_df, create_frame, create_master_df, create_master_player_min_df
import string

# players to match up data with

In [13]:
games = db.games.find()
players_minutes_df = create_master_player_min_df(games)

In [244]:
players_minutes_df['squad_number'].unique()

array([22., 12., 24.,  6., 29., 34.,  4., 15., 16., 30., 28.,  8., 23.,
       10., 11., 26.,  9., 33., nan, 13., 20., 21.,  5., 18., 31.,  7.,
       19.,  3., 14., 25.,  2., 27., 41., 40., 37., 32.,  1., 17., 39.,
       38., 87., 35., 36., 42., 43.])

In [245]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0


In [247]:
names = players_minutes_df['name'].unique()

In [291]:
afa_team_dict = {20: 'VEL', 13: 'NOB', 136: 'TIG', 19: 'SLO', 8: 'GIM', 2: 'ARG',
 137: 'UNI', 122: 'ALD', 869: 'PA', 6: 'COL', 124: 'BEL', 134: 'SMS',
 5: 'BOC', 135: 'TAL', 132: 'GOD', 7: 'EST', 12: 'LAN', 129: 'DEF',
 18: 'ROS', 4: 'BAN', 100: 'HUR', 17: 'RIV', 815: 'ATT', 16: 'RAC', 10: 'IND',
 490: 'SMT'}

In [292]:
len(afa_team_dict)

26

In [262]:
team_dict = [{'team_id': '20', 'team_intitials': 'VEL', 'short_name': 'Vélez'},
 {'team_id': '13', 'team_intitials': 'NOB', 'short_name': 'Newell`s'},
 {'team_id': '136', 'team_intitials': 'TIG', 'short_name': 'Tigre'},
 {'team_id': '19', 'team_intitials': 'SLO', 'short_name': 'San Lorenzo'},
 {'team_id': '8', 'team_intitials': 'GIM', 'short_name': 'Gimnasia'},
 {'team_id': '2', 'team_intitials': 'ARG', 'short_name': 'Argentinos'},
 {'team_id': '137', 'team_intitials': 'UNI', 'short_name': 'Unión'},
 {'team_id': '122', 'team_intitials': 'ALD', 'short_name': 'Aldosivi'},
 {'team_id': '869', 'team_intitials': 'PA', 'short_name': 'Patronato'},
 {'team_id': '6', 'team_intitials': 'COL', 'short_name': 'Colón'},
 {'team_id': '124', 'team_intitials': 'BEL', 'short_name': 'Belgrano'},
 {'team_id': '134', 'team_intitials': 'SMS', 'short_name': 'S. Martín SJ'},
 {'team_id': '5', 'team_intitials': 'BOC', 'short_name': 'Boca'},
 {'team_id': '135', 'team_intitials': 'TAL', 'short_name': 'Talleres'},
 {'team_id': '132', 'team_intitials': 'GOD', 'short_name': 'Godoy Cruz'},
 {'team_id': '7', 'team_intitials': 'EST', 'short_name': 'Estudiantes'},
 {'team_id': '12', 'team_intitials': 'LAN', 'short_name': 'Lanús'},
 {'team_id': '129', 'team_intitials': 'DEF', 'short_name': 'Defensa'},
 {'team_id': '18', 'team_intitials': 'ROS', 'short_name': 'R. Central'},
 {'team_id': '4', 'team_intitials': 'BAN', 'short_name': 'Banfield'},
 {'team_id': '100', 'team_intitials': 'HUR', 'short_name': 'Huracán'},
 {'team_id': '17', 'team_intitials': 'RIV', 'short_name': 'River'},
 {'team_id': '815', 'team_intitials': 'ATT', 'short_name': 'Atl. Tucumán'},
 {'team_id': '16', 'team_intitials': 'RAC', 'short_name': 'Racing'},
 {'team_id': '10', 'team_intitials': 'IND', 'short_name': 'Independiente'},
 {'team_id': '490', 'team_intitials': 'SMT', 'short_name': 'S. Martín T'}]

In [293]:
players_minutes_df['club_brev'] = players_minutes_df['team_id'].map(afa_team_dict)

In [295]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played,club_brev
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0,VEL
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0,VEL
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0,VEL
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0,VEL
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0,VEL


# Transfer Market Data

In [307]:
players = db.players.find()

In [308]:
master_list = []
for player in players:
    master_list.append(player)

In [309]:
tm_player = pd.DataFrame(master_list)


In [310]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18,£5.40m


### Turn squad numbers into floats...

In [311]:
tm_player['squad_num'].unique()

array(['31', '28', '6', '2', '18', '20', '29', '16', '15', '8', '30', '7',
       '22', '19', '17', '1', '25', '24', '26', '3', '14', '4', '40', '5',
       '39', '10', '11', '32', '9', '13', '-', '23', '21', '27', '41',
       '34', '37', '12', '36', '38', '33', '35', '87'], dtype=object)

In [312]:
tm_player.loc[tm_player.squad_num == '-', 'squad_num'] = None

In [313]:
tm_player['squad_num'].unique()

array(['31', '28', '6', '2', '18', '20', '29', '16', '15', '8', '30', '7',
       '22', '19', '17', '1', '25', '24', '26', '3', '14', '4', '40', '5',
       '39', '10', '11', '32', '9', '13', None, '23', '21', '27', '41',
       '34', '37', '12', '36', '38', '33', '35', '87'], dtype=object)

In [314]:
tm_player = tm_player.astype({'squad_num': float})

In [316]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31.0,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28.0,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6.0,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2.0,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18.0,£5.40m


### Add Column of Club_Abbrev

In [265]:
tm_player['club'].unique()

array(['Squad Club Atlético Boca Juniors',
       'Squad Club Atlético Independiente',
       'Squad Club Atlético River Plate', 'Squad Racing Club',
       'Squad Club Atlético San Lorenzo de Almagro',
       'Squad Club Atlético Lanús', 'Squad Club Atlético Vélez Sarsfield',
       'Squad Club Deportivo Godoy Cruz Antonio Tomba',
       'Squad Club Atlético Colón',
       'Squad Club Social y Deportivo Defensa y Justicia',
       'Squad Club Atlético Rosario Central',
       'Squad Club Atlético Talleres',
       'Squad Club Atlético Unión (Santa Fe)',
       'Squad Club Estudiantes de La Plata',
       'Squad Club Atlético Huracán',
       'Squad Asociación Atlética Argentinos Juniors',
       'Squad Club Atlético Banfield', 'Squad Club Atlético Belgrano',
       'Squad Club Atlético Tigre',
       "Squad Club Atlético Newell's Old Boys",
       'Squad Club Atlético Tucumán',
       'Squad Club de Gimnasia y Esgrima La Plata',
       'Squad Club Atlético Aldosivi',
       'Squad Clu

In [268]:
tm_club_brev = {'Squad Club Atlético Boca Juniors': 'BOC' , 'Squad Club Atlético Independiente': 'IND', 'Squad Club Atlético River Plate': 'RIV', 'Squad Racing Club': 'RAC',
       'Squad Club Atlético San Lorenzo de Almagro': 'SLO', 'Squad Club Atlético Lanús': 'LAN', 'Squad Club Atlético Vélez Sarsfield': 'VEL',
       'Squad Club Deportivo Godoy Cruz Antonio Tomba': 'GOD', 'Squad Club Atlético Colón': 'COL', 'Squad Club Social y Deportivo Defensa y Justicia': 'DEF',
       'Squad Club Atlético Rosario Central': 'ROS', 'Squad Club Atlético Talleres': 'TAL', 'Squad Club Atlético Unión (Santa Fe)': 'UNI',
       'Squad Club Estudiantes de La Plata': 'EST', 'Squad Club Atlético Huracán': 'HUR', 'Squad Asociación Atlética Argentinos Juniors': 'ARG',
       'Squad Club Atlético Banfield': 'BAN', 'Squad Club Atlético Belgrano': 'BEL', 'Squad Club Atlético Tigre': 'TIG', "Squad Club Atlético Newell's Old Boys": 'NOB',
       'Squad Club Atlético Tucumán': 'ATT', 'Squad Club de Gimnasia y Esgrima La Plata': 'GIM', 'Squad Club Atlético Aldosivi': 'ALD',
       'Squad Club Atlético San Martín (SJ)': 'SMS', 'Squad Club Atlético San Martín (Tucumán)': 'SMT', 'Squad Club Atlético Patronato de la Juventud Católica': 'PA'}

In [269]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18,£5.40m


In [317]:
tm_player['club_brev'] = tm_player['club'].map(tm_club_brev)

In [318]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings),club_brev
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31.0,£4.50m,BOC
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28.0,£495k,BOC
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6.0,£4.05m,BOC
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2.0,£1.62m,BOC
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18.0,£5.40m,BOC


## Test Merge

In [321]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played,club_brev
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0,VEL
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0,VEL
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0,VEL
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0,VEL
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0,VEL


In [323]:
players = players_minutes_df['player_id'].unique()
player_minutes = []
for player in players:
    total_minutes = players_minutes_df[players_minutes_df['player_id'] == player]['minutes_played'].sum()
    name = players_minutes_df[players_minutes_df['player_id'] == player]['name'].iloc[0]
    squad_num = players_minutes_df[players_minutes_df['player_id'] == player]['squad_number'].iloc[0]
    club_brev = players_minutes_df[players_minutes_df['player_id'] == player]['club_brev'].iloc[0]
    player_minutes.append([player, total_minutes, name, squad_num, club_brev])
    
summed_player_min = pd.DataFrame(player_minutes, columns=['player_id', 'total_minutes_played', 'player_name', 'squad_num', 'club_brev'])


In [325]:
summed_player_min.head()

,player_id,total_minutes_played,player_name,squad_num,club_brev
0,19132.0,1080.000000,Alexander Domínguez,22.0,VEL
1,27987.0,90.000000,Lucas Hoyos,12.0,VEL
2,30896.0,896.483333,Gastón Díaz,24.0,VEL
3,94225.0,990.000000,Joaquín Laso,6.0,VEL
4,99779.0,1080.000000,Luis Abram,29.0,VEL


In [327]:
player_merge_df = pd.merge(summed_player_min, tm_player, on=['club_brev', 'squad_num'])

In [329]:
player_merge_df.drop(columns=['_id', 'club_brev'], inplace=True)

In [335]:
columns = ['player_id', 'player_name', 'foot', 'birthday', 'club', 'transfer_value(sterlings)', 'total_minutes_played']

In [336]:
player_merge_df[columns]

,player_id,player_name,foot,birthday,club,transfer_value(sterlings),total_minutes_played
0,19132.0,Alexander Domínguez,right,"Jun 5, 1987 (31)",Squad Club Atlético Vélez Sarsfield,£1.58m,1080.000000
1,27987.0,Lucas Hoyos,right,"Apr 29, 1989 (29)",Squad Club Atlético Vélez Sarsfield,£135k,90.000000
2,30896.0,Gastón Díaz,right,"Mar 13, 1988 (30)",Squad Club Atlético Vélez Sarsfield,£450k,896.483333
3,94225.0,Joaquín Laso,right,"Jul 4, 1990 (28)",Squad Club Atlético Vélez Sarsfield,£270k,990.000000
4,99779.0,Luis Abram,left,"Feb 27, 1996 (22)",Squad Club Atlético Vélez Sarsfield,£405k,1080.000000
5,208534.0,Francisco Ortega,left,"Feb 19, 1999 (19)",Squad Club Atlético Vélez Sarsfield,£360k,360.000000
6,208462.0,Hernán De La Fuente,right,"Jan 7, 1997 (21)",Squad Club Atlético Vélez Sarsfield,£540k,270.000000
7,193952.0,Nahuel Arena,right,"Jun 2, 1998 (20)",Squad Club Atlético Vélez Sarsfield,-,-1.450000
8,185745.0,Lucas Robertone,right,"Mar 18, 1997 (21)",Squad Club Atlético Vélez Sarsfield,£2.70m,881.500000
9,60730.0,Gastón Giménez,left,"Jul 27, 1991 (27)",Squad Club Atlético Vélez Sarsfield,£1.80m,1126.450000
